In [67]:
import re
import csv

In [69]:
# le nettoyege de notre data pour etre pret de traiter
def nettoyer_texte(texte):
    # Conserver uniquement les lettres arabes, les diacritiques (tachkīl) et les espaces
    # Supprimer les chiffres, la ponctuation (y compris la virgule et le point-virgule arabes), etc.
    texte_nettoye = re.sub(r'[^\u0600-\u06FF\u0610-\u061A\u064B-\u065F\u0670\u06D6-\u06DC\u06DF-\u06E8\u06EA-\u06ED\s]', '', texte)
    # Supprimer spécifiquement la virgule arabe (،) et le point-virgule arabe (؛)
    texte_nettoye = re.sub(r'[،؟؛]', '', texte_nettoye)
    return texte_nettoye

def supprimer_caractere_apres_chadda(texte):
    # Parcourir le texte pour supprimer le caractère qui suit la chadda
    texte_modifie = []
    i = 0
    while i < len(texte):
        if i + 1 < len(texte) and texte[i] == '\u0651':  # Si le caractère est une chadda
            texte_modifie.append(texte[i])  # Ajouter la chadda
            i += 2  # Passer la chadda et le caractère suivant
        else:
            texte_modifie.append(texte[i])  # Ajouter le caractère actuel
            i += 1
    return ''.join(texte_modifie)

def ajouter_sukun(texte):
    # Liste des diacritiques (tachkīl) en arabe
    diacritiques = set(['\u064B', '\u064C', '\u064D', '\u064E', '\u064F', '\u0650', '\u0651', '\u0652'])  # Fatha, Damma, Kasra, Sukūn, Chadda, etc.
    texte_modifie = []
    i = 0
    while i < len(texte):
        caractere = texte[i]
        # Si le caractère est une lettre arabe
        if '\u0600' <= caractere <= '\u06FF':
            # Vérifier si la lettre n'a pas de tachkīl
            if i + 1 >= len(texte) or texte[i + 1] not in diacritiques:
                texte_modifie.append(caractere)
                texte_modifie.append('\u0652')  # Ajouter un sukun
                i += 1
            else:
                # Conserver la lettre et son tachkīl
                texte_modifie.append(caractere)
                texte_modifie.append(texte[i + 1])
                i += 2
        else:
            # Conserver les autres caractères (espaces, etc.)
            texte_modifie.append(caractere)
            i += 1
    return ''.join(texte_modifie)

def supprimer_caracteres_successifs(texte):
    # Parcourir le texte pour supprimer un caractère sur deux dans les paires successives
    texte_modifie = []
    i = 0
    while i < len(texte):
        caractere = texte[i]
        texte_modifie.append(caractere)  # Ajouter le premier caractère de la paire
        i += 2  # Passer au caractère suivant après la paire
    return ''.join(texte_modifie)

In [71]:
# creation d'une matrice pour les transition
def creer_matrice_transition(texte):
    # Liste des diacritiques (tachkīl) en arabe
    diacritiques = set(['\u064B', '\u064C', '\u064D', '\u064E', '\u064F', '\u0650', '\u0651', '\u0652'])  # Fatha, Damma, Kasra, Sukūn, Chadda, etc.
    
    # Initialiser la matrice de transition
    matrice = {}
    
    # Parcourir le texte pour remplir la matrice
    i = 0
    while i < len(texte) - 1:
        # Extraire la lettre actuelle et son tachkīl
        lettre_actuelle = texte[i]
        tachkil_actuel = texte[i + 1] if i + 1 < len(texte) and texte[i + 1] in diacritiques else ''
        
        # Extraire la lettre suivante et son tachkīl
        j = i + 1 + (1 if tachkil_actuel else 0)
        while j < len(texte) and texte[j].isspace():  # Ignorer les espaces pour le caractère suivant
            j += 1
        lettre_suivante = texte[j] if j < len(texte) else ''
        tachkil_suivant = texte[j + 1] if j + 1 < len(texte) and texte[j + 1] in diacritiques else ''
        
        # Créer les clés pour la matrice
        cle_actuelle = lettre_actuelle + (tachkil_actuel if tachkil_actuel else '')
        cle_suivante = lettre_suivante + (tachkil_suivant if tachkil_suivant else '')
        
        # Mettre à jour la matrice uniquement si le caractère suivant n'est pas un espace
        if not lettre_suivante.isspace():
            if cle_actuelle not in matrice:
                matrice[cle_actuelle] = {}
            if cle_suivante not in matrice[cle_actuelle]:
                matrice[cle_actuelle][cle_suivante] = 0
            matrice[cle_actuelle][cle_suivante] += 1
        
        # Passer à la prochaine lettre
        i = j if j > i else i + 1
    
    return matrice

In [73]:
#Calcule de la probabilite de chaque transition
def calculer_probabilites(matrice):
    # Calculer les probabilités pour chaque ligne de la matrice
    for lettre_actuelle in matrice:
        total = sum(matrice[lettre_actuelle].values())  # Total des occurrences pour la lettre actuelle
        if total > 0:
            for lettre_suivante in matrice[lettre_actuelle]:
                matrice[lettre_actuelle][lettre_suivante] /= total  # Remplacer l'occurrence par la probabilité
    return matrice

In [75]:
# Enregistrer les resultat obtenue dans un fichier csv pour etre simple a visualiser
def enregistrer_matrice_csv(matrice, chemin_csv):
    # Extraire toutes les lettres avec tachkīl pour les en-têtes
    lettres = sorted(set(matrice.keys()).union(*[set(matrice[k].keys()) for k in matrice]))
    
    # Filtrer les clés qui ne contiennent que des tachkīl (sans lettre arabe)
    lettres_filtrees = [lettre for lettre in lettres if any(caractere not in set(['\u064B', '\u064C', '\u064D', '\u064E', '\u064F', '\u0650', '\u0651', '\u0652']) for caractere in lettre)]
    
    # Ouvrir le fichier CSV en mode écriture
    with open(chemin_csv, 'w', encoding='utf-8', newline='') as fichier_csv:
        writer = csv.writer(fichier_csv)
        
        # Écrire l'en-tête
        writer.writerow([''] + lettres_filtrees)
        
        # Écrire les lignes de la matrice
        for lettre_actuelle in lettres_filtrees:
            ligne = [lettre_actuelle]
            for lettre_suivante in lettres_filtrees:
                if lettre_actuelle in matrice and lettre_suivante in matrice[lettre_actuelle]:
                    ligne.append(f"{matrice[lettre_actuelle][lettre_suivante]:.4f}")  # Formater la probabilité avec 4 décimales
                else:
                    ligne.append("0.0000")  # Probabilité nulle si la transition n'existe pas
            writer.writerow(ligne)


In [77]:
# Programme pricipale 
def programme_principale(chemin_entree, chemin_sortie,chemin_csv):
    # Lire le fichier d'entrée
    with open(chemin_entree, 'r', encoding='utf-8') as fichier_entree:
        contenu = fichier_entree.read()
    
    # Nettoyer le texte
    contenu_nettoye = nettoyer_texte(contenu)
    
    # Supprimer le caractère qui suit la chadda
    contenu_modifie = supprimer_caractere_apres_chadda(contenu_nettoye)
    
    # Ajouter un sukun aux lettres sans tachkīl
    contenu_modifie = ajouter_sukun(contenu_modifie)

    # Créer la matrice de transition
    matrice = creer_matrice_transition(contenu_nettoye)
    
    # Calculer les probabilités de transition
    matrice = calculer_probabilites(matrice)
    
    # Afficher la matrice
    enregistrer_matrice_csv(matrice, chemin_csv)
    
    # Écrire le résultat dans le fichier de sortie
    with open(chemin_sortie, 'w', encoding='utf-8') as fichier_sortie:
        fichier_sortie.write(contenu_modifie)


In [79]:
# Exemple d'utilisation
chemin_entree = 'data_tachkil_arabe.txt'  # votre fichier d'entrée
chemin_sortie = 'fichier_sortie.txt'  # votre fichier de sortie
chemin_csv = 'matrice_transition.csv' # fichier csv pour enregistrer les transition
programme_principale(chemin_entree, chemin_sortie,chemin_csv)
print("Le traitement et faite et enregistré avec succès.")

NameError: name 'programme_principal' is not defined